# Population by commune

In [1]:
from pynsee.geodata import get_geodata_list, get_geodata

import math
import pandas as pd
from pandas.api.types import CategoricalDtype
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import descartes

/usr/local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import logging
import sys
logging.basicConfig(stream=sys.stdout,
                    level=logging.INFO, 
                    format="%(message)s")

In [3]:
# get geographical data list
geodata_list = get_geodata_list()
# get departments geographical limits
com = get_geodata('ADMINEXPRESS-COG-CARTO.LATEST:commune', update=True).to_crs("EPSG:3035")

Previously saved data has been used:
/home/onyxia/.cache/pynsee/pynsee/e879822687bab4b12f23762b5d56742b.parquet
Creation date: 2025-06-23, today
Set update=True to get the most up-to-date data
INSEE API credentials have not been found: please try to reuse pynsee.init_conn to save them locally.
Otherwise, you can still use environment variables as follow:

import os
os.environ['sirene_key'] = 'my_sirene_key'
https://data.geopf.fr/wfs/ows?service=WFS&version=2.0.0&request=GetFeature&typenames=ADMINEXPRESS-COG-CARTO.LATEST:commune&resultType=hits
https://data.geopf.fr/wfs/?service=WFS&version=2.0.0&request=GetFeature&typenames=ADMINEXPRESS-COG-CARTO.LATEST:commune&outputFormat=application/json&startIndex=0&count=5000&srsName=EPSG:3857
https://data.geopf.fr/wfs/?service=WFS&version=2.0.0&request=GetFeature&typenames=ADMINEXPRESS-COG-CARTO.LATEST:commune&outputFormat=application/json&startIndex=10000&count=5000&srsName=EPSG:3857
https://data.geopf.fr/wfs/?service=WFS&version=2.0.0&request=G

100%|██████████| 7/7 [00:09<00:00,  1.31s/it]


Data saved:
/home/onyxia/.cache/pynsee/pynsee/6cb5dc200198fcdb00222fe8b8f7945b.parquet


In [4]:
com[:2]

,geometry,cleabs,nom_officiel,nom_officiel_en_majuscules,statut,code_insee,population,date_du_recensement,organisme_recenseur,code_insee_du_canton,code_insee_de_l_arrondissement,code_insee_du_departement,code_insee_de_la_region,code_siren,codes_siren_des_epci,code_postal,superficie_cadastrale
0,"MULTIPOLYGON (((3931418.575 2573235.55, 393139...",COMMUNE_0000000000001001,L'Abergement-Clémenciat,L'ABERGEMENT-CLEMENCIAT,Commune simple,01001,859,2022-01-01Z,INSEE,0108,012,01,84,210100012,200069193,01400,1590
1,"MULTIPOLYGON (((3966704.772 2551978.874, 39666...",COMMUNE_0000000000001002,L'Abergement-de-Varey,L'ABERGEMENT-DE-VAREY,Commune simple,01002,273,2022-01-01Z,INSEE,0101,011,01,84,210100020,240100883,01640,920


In [6]:
com["area"] = com['geometry'].area / 10**6
mapcom = com.to_crs(epsg=3857)

mapcom['REF_AREA'] = 'D' + mapcom['code_insee_du_departement']
mapcom['density'] = mapcom['population'] / mapcom['area'] 

In [7]:
mapcom = mapcom.transform_overseas(
    departement=['971', '972', '974', '973', '976'],  factor=[1.5, 1.5, 1.5, 0.35, 1.5])

https://data.geopf.fr/wfs/ows?service=WFS&version=2.0.0&request=GetFeature&typenames=ADMINEXPRESS-COG-CARTO.LATEST:departement&resultType=hits
https://data.geopf.fr/wfs/?service=WFS&version=2.0.0&request=GetFeature&typenames=ADMINEXPRESS-COG-CARTO.LATEST:departement&outputFormat=application/json&startIndex=0&count=101&srsName=EPSG:3857
Data saved:
/home/onyxia/.cache/pynsee/pynsee/f2bbdea5546ef02791b1286f1b6b1a51.parquet


ValueError: Could not get department geometries.

In [ ]:
mapcom = mapcom.zoom(departement = ["75","92", "93", "91", "77", "78", "95", "94"],
                     factor=1.5, startAngle = math.pi * (1 - 3 * 1/9))

In [ ]:
mapcom.loc[mapcom.density < 40, 'range'] = "< 40"
mapcom.loc[mapcom.density >= 20000, 'range'] = "> 20 000"

density_ranges = [40, 80, 100, 120, 150, 200, 250, 400, 600, 1000, 2000, 5000, 10000, 20000]
list_ranges = []
list_ranges.append( "< 40")

for i in range(len(density_ranges)-1):
    min_range = density_ranges[i]
    max_range = density_ranges[i+1]
    range_string = "[{}, {}[".format(min_range, max_range)
    mapcom.loc[(mapcom.density >= min_range) & (mapcom.density < max_range), 'range'] = range_string
    list_ranges.append(range_string)

list_ranges.append("> 20 000")

mapcom['range'] = mapcom['range'].astype(CategoricalDtype(categories=list_ranges, ordered=True))  

In [ ]:
fig, ax = plt.subplots(1,1,figsize=[15,15])
mapcom.plot(column='range', cmap=cm.viridis,
    legend=True, ax=ax,
    legend_kwds={'bbox_to_anchor': (1.1, 0.8),
                 'title':'density per km2'})
ax.set_axis_off()
ax.set(title='Distribution of population in France')
plt.show()

fig.savefig('pop_france.svg',
            format='svg', dpi=1200,
            bbox_inches = 'tight',
            pad_inches = 0)